### NNX is currently in an experimental state and is subject to change. Linen is still the recommended option for large-scale projects. Feedback and contributions are welcome!

### NNX 기본 사항

NNX는 JAX를 위한 신경망 라이브러리로, 최고의 개발 경험을 제공하는 데 중점을 두고 있습니다. 따라서 신경망을 구축하고 실험하는 것이 쉽고 직관적입니다. NNX는 PyTrees 대신 PyGraphs로 객체를 표현하여 reference 공유와 mutability를 가능하게 합니다. 이 설계는 사용자 모델이 친숙한 Python 객체 지향 코드와 유사하게 만들어, 특히 PyTorch와 같은 프레임워크 사용자에게 매력적입니다.

간단한 구현에도 불구하고 NNX는 Linen이 대형 코드베이스로 효과적으로 확장할 수 있도록 해준 것과 동일한 강력한 디자인 패턴을 지원합니다.

In [1]:
from flax import nnx
import jax
import jax.numpy as jnp

### 모듈 시스템

먼저 NNX를 사용하여 `Linear` 모듈을 만드는 방법을 알아봅시다. NNX와 Haiku 또는 Linen과 같은 모듈 시스템의 주요 차이점은 NNX에서 모든 것이 명시적이라는 점입니다. 이는 다음을 의미합니다: 1) 모듈 자체가 상태(예: 매개변수)를 직접 보유하고, 2) RNG 상태는 사용자가 스레드로 처리하며, 3) 모든 shape 정보는 초기화 시 제공되어야 합니다(no shape inference).

다음과 같이 동적 상태는 보통 `nnx.Param`에 저장되고, 정적 상태(NNX가 처리하지 않는 모든 유형, 예를 들어 정수 또는 문자열)는 직접 저장됩니다. `jax.Array` 및 `numpy.ndarray` 유형의 속성도 동적 상태로 취급되지만, 이를 `nnx.Variable`(예: `Param`)에 저장하는 것이 더 바람직합니다. 또한, `nnx.Rngs` 객체를 사용하여 생성자에 전달된 루트 키를 기반으로 새로운 고유 키를 얻을 수 있습니다.

In [2]:
class Linear(nnx.Module):
  def __init__(self, din: int, dout: int, *, rngs: nnx.Rngs):
    key = rngs.params()
    self.w = nnx.Param(jax.random.uniform(key, (din, dout)))
    self.b = nnx.Param(jnp.zeros((dout,)))
    self.din, self.dout = din, dout

  def __call__(self, x: jax.Array):
    return x @ self.w + self.b

`nnx.Variable`의 내부 값은 `.value` 속성을 사용하여 접근할 수 있습니다. 그러나 편의를 위해 모든 숫자 연산자를 구현하였으며, 산술 표현식에서 직접 사용할 수 있습니다(위에 표시된 것처럼). 또한, Variables는 `__jax_array__` 프로토콜을 구현하므로(내부 값이 JAX 배열인 경우) 모든 JAX 함수에 전달될 수 있습니다.

실제로 모듈을 초기화하려면 생성자를 호출하기만 하면 됩니다. 모듈의 모든 매개변수는 보통 즉시 생성됩니다. 모듈이 자체 상태 메서드를 보유하고 있기 때문에 별도의 `apply` 메서드를 필요로 하지 않고 직접 호출할 수 있습니다. 이는 모델의 전체 구조를 직접 검사할 수 있기 때문에 디버깅에 매우 편리합니다.

In [3]:
model = Linear(2, 5, rngs=nnx.Rngs(params=0))
y = model(x=jnp.ones((1, 2)))

print(y)
nnx.display(model)

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


[[1.245453   0.74195766 0.8553282  0.6763327  1.2617068 ]]


## Stateful Computation

BatchNorm과 같은 레이어를 구현하려면 forward pass 동안 상태 업데이트를 수행해야 합니다. NNX에서 이를 구현하려면 `Variable`을 생성하고 forward pass 동안 해당 `.value`를 업데이트하기만 하면 됩니다.

In [4]:
class Count(nnx.Variable): pass

class Counter(nnx.Module):
  def __init__(self):
    self.count = Count(jnp.array(0))

  def __call__(self):
    self.count += 1

counter = Counter()
print(f'{counter.count.value = }')
counter()
print(f'{counter.count.value = }')

counter.count.value = Array(0, dtype=int32, weak_type=True)
counter.count.value = Array(1, dtype=int32, weak_type=True)


Mutable references are usually avoided in JAX, however as we’ll see in later sections NNX provides sound mechanisms to handle them.

### Nested Modules

예상한 대로, 모듈은 중첩된 구조에서 다른 모듈을 구성하는 데 사용할 수 있으며, 이는 attribute로 직접 할당하거나, (중첩된) pytree 유형의 attribute 내에 할당할 수 있습니다. 예: `list`, `dict`, `tuple` 등. 아래 예에서는 두 개의 `Linear` 레이어, 하나의 `Dropout` 레이어 및 하나의 `BatchNorm` 레이어로 구성된 간단한 `MLP` 모듈을 정의합니다.

In [5]:
class MLP(nnx.Module):
  def __init__(self, din: int, dmid: int, dout: int, *, rngs: nnx.Rngs):
    self.linear1 = Linear(din, dmid, rngs=rngs)
    self.dropout = nnx.Dropout(rate=0.1, rngs=rngs)
    self.bn = nnx.BatchNorm(dmid, rngs=rngs)
    self.linear2 = Linear(dmid, dout, rngs=rngs)

  def __call__(self, x: jax.Array):
    x = nnx.gelu(self.dropout(self.bn(self.linear1(x))))
    return self.linear2(x)
  
model = MLP(2, 16, 5, rngs=nnx.Rngs(0))

y = model(x=jnp.ones((3, 2)))

nnx.display(model)

In NNX `Dropout` is a stateful module that stores an `Rngs` object so that it can generate new masks during the forward pass without the need for the user to pass a new key each time.

### Model Surgery

NNX 모듈은 기본적으로 mutable합니다. 이는 구조를 언제든지 변경할 수 있음을 의미합니다. 따라서 모델 수술이 매우 간단해지며, 어떤 서브모듈 attribute도 다른 것으로 대체할 수 있습니다. 예: 새로운 모듈, 기존 공유 모듈, 다른 유형의 모듈 등. 또한, `Variable`도 수정, 대체 또는 공유할 수 있습니다.

다음 예는 이전의 `MLP` 모델에서 `Linear` 레이어를 `LoraLinear` 레이어로 교체하는 방법을 보여줍니다.

```python
class MLP(nnx.Module):
  def __init__(self, din: int, dmid: int, dout: int, *, rngs: nnx.Rngs):
    self.linear1 = Linear(din, dmid, rngs=rngs)
    self.dropout = nnx.Dropout(rate=0.1, rngs=rngs)
    self.bn = nnx.BatchNorm(dmid, rngs=rngs)
    self.linear2 = Linear(dmid, dout, rngs=rngs)

  def __call__(self, x: jax.Array):
    x = nnx.gelu(self.dropout(self.bn(self.linear1(x))))
    return self.linear2(x)
```

In [6]:
class LoraParam(nnx.Param): pass

class LoraLinear(nnx.Module):
  def __init__(self, linear: Linear, rank: int, rngs: nnx.Rngs):
    self.linear = linear
    #                                 linear1 : (2, 4)  / linear2 : (32, 4)
    self.A = LoraParam(jax.random.normal(rngs(), (linear.din, rank)))
    self.B = LoraParam(jax.random.normal(rngs(), (rank, linear.dout)))
    #                                 linear1 : (4, 32) / linear2 : (4, 5)

  def __call__(self, x: jax.Array):
    # self.linear(x) + (B, din) @ (din, rank) @ (rank, dout)
    return self.linear(x) + x @ self.A @ self.B

rngs = nnx.Rngs(0)
model = MLP(2, 32, 5, rngs=rngs)
# din, dmid, dout, rngs
# model.linear1 : (2, 32)
# model.linear2 : (32, 5)


# model surgery
model.linear1 = LoraLinear(model.linear1, 4, rngs=rngs)
model.linear2 = LoraLinear(model.linear2, 4, rngs=rngs)

y = model(x=jnp.ones((3, 2)))

nnx.display(model)

### NNX Transforms

NNX Transforms는 JAX transform을 확장하여 모듈 및 기타 객체를 지원합니다. 이는 객체의 상태를 인식하고 이를 변환하기 위한 추가 API를 제공하는 등 동등한 JAX 대응보다 우수합니다. NNX Transforms의 주요 기능 중 하나는 참조 의미 체계의 보존(preservation of reference semantics)입니다. 이는 변환 내에서 발생하는 객체 그래프의 변경이 변환 규칙 내에서 합법적인 한(its legal within the transform rules) 외부로 전파됨을 의미합니다. 실제로 이는 NNX 프로그램이 명령형(imperative) 코드를 사용하여 표현될 수 있음을 의미하며, 사용자 경험을 크게 단순화합니다.

다음 예제에서는 `train_step` 함수를 정의합니다. 이 함수는 `MLP` 모델, `Optimizer`, 데이터 배치를 받아 해당 단계의 loss을 반환합니다. loss과 gradient는 `nnx.value_and_grad` 변환을 사용하여 `loss_fn`에서 계산됩니다. 기울기는 옵티마이저의 `update` 메서드에 전달되어 모델의 매개변수를 업데이트합니다.

```python
class MLP(nnx.Module):
  def __init__(self, din: int, dmid: int, dout: int, *, rngs: nnx.Rngs):
    self.linear1 = Linear(din, dmid, rngs=rngs)
    self.dropout = nnx.Dropout(rate=0.1, rngs=rngs)
    self.bn = nnx.BatchNorm(dmid, rngs=rngs)
    self.linear2 = Linear(dmid, dout, rngs=rngs)

  def __call__(self, x: jax.Array):
    x = nnx.gelu(self.dropout(self.bn(self.linear1(x))))
    return self.linear2(x)
```

In [7]:
import optax

# MLP contains 2 Linear layers, 1 Dropout layer, 1 BatchNorm layer
model = MLP(2, 16, 10, rngs=nnx.Rngs(0))
optimizer = nnx.Optimizer(model, optax.adam(1e-3))  # reference sharing

@nnx.jit  # automatic state management, 외부의 `model` reference까지 전파
def train_step(model, optimizer, x, y):
  def loss_fn(model: MLP):
    y_pred = model(x)
    return jnp.mean((y_pred - y) ** 2)

  loss, grads = nnx.value_and_grad(loss_fn)(model)
  optimizer.update(grads)  # inplace updates

  return loss

x, y = jnp.ones((5, 2)), jnp.ones((5, 10))
loss = train_step(model, optimizer, x, y)

print(f'{loss = }')
print(f'{optimizer.step.value = }')

loss = Array(1.0000278, dtype=float32)
optimizer.step.value = Array(1, dtype=uint32)


이 예제에서 주목할 만한 몇 가지 사항이 있습니다:

1. `BatchNorm` 및 `Dropout` 레이어 상태의 업데이트는 `loss_fn` 내에서 `train_step`을 거쳐 외부의 `model` reference 까지 자동으로 전파됩니다.
2. `optimizer`는 `model`에 대한 mutable reference를 보유하고 있으며, 이 관계는 `train_step` 함수 내에서 보존되어 옵티마이저만으로 모델의 매개변수를 업데이트할 수 있게 합니다.

### Scan over layers

다음으로, `nnx.vmap`을 사용하여 `MLP` 스택을 만들고 `nnx.scan`을 사용하여 스택의 각 레이어를 입력에 반복적으로 적용하는 예제를 살펴보겠습니다 (레이어를 스캔).

다음 사항에 주목하세요:

1. `create_model` 함수는 (단일) `MLP` 객체를 생성하며, 이는 `nnx.vmap`에 의해 추가적인 크기 `axis_size`의 차원을 가지도록 리프팅됩니다.
2. `forward` 함수는 `MLP` 객체의 상태를 인덱싱하여 각 단계에서 다른 매개변수 집합(a different set of parameters)을 가져옵니다.
3. `nnx.scan`은 `forward` 내에서 `BatchNorm` 및 `Dropout` 레이어의 상태 업데이트를 자동으로 `model` 참조로 외부에 전파합니다.

In [8]:
from functools import partial

@partial(nnx.vmap, axis_size=5)
def create_model(rngs: nnx.Rngs):
  return MLP(10, 32, 10, rngs=rngs)

model = create_model(nnx.Rngs(0))

@nnx.scan
def forward(x, model: MLP):
  x = model(x)
  return x, None

x = jnp.ones((3, 10))
y, _ = forward(x, model)

print(f'{y.shape = }')
nnx.display(model)

y.shape = (3, 10)


How do NNX transforms achieve this? To understand how NNX objects interact with JAX transforms lets take a look at the Functional API.

### The Functional API

Functional API는 참조/객체 의미 체계와 값/pytree 의미 체계 간의 clear boundary를 설정합니다. 또한 Linen/Haiku 사용자들이 익숙한 상태에 대한 세밀한 제어를 동일하게 허용합니다. Functional API는 세 가지 기본 메서드로 구성됩니다: `split`, `merge`, 그리고 `update`.

아래에 표시된 `StatefulLinear` 모듈은 Functional API 사용 예제로 제공됩니다. 이 모듈은 몇 가지 `nnx.Param` 변수와 각 forward pass에서 증가하는 정수 스칼라 상태를 추적하는 데 사용되는 사용자 정의 `Count` 변수 유형을 포함합니다.

In [15]:
class Count(nnx.Variable): pass

class StatefulLinear(nnx.Module):
  def __init__(self, din: int, dout: int, *, rngs: nnx.Rngs):
    self.w = nnx.Param(jax.random.uniform(rngs(), (din, dout)))
    self.b = nnx.Param(jnp.zeros((dout,)))
    self.count = Count(jnp.array(0, dtype=jnp.uint32))

  def __call__(self, x: jax.Array):
    self.count += 1
    return x @ self.w + self.b

  #     (B, 3) -> (B, 5)
model = StatefulLinear(din=3, dout=5, rngs=nnx.Rngs(0))
y = model(jnp.ones((1, 3)))

print(f'{y.shape = }')
nnx.display(model)

y.shape = (1, 5)


### State and GraphDef

모듈은 `split` 함수를 사용하여 `GraphDef`와 `State`로 분해할 수 있습니다. State는 문자열에서 Variables 또는 nested States로의 매핑입니다. GraphDef는 모듈 그래프를 재구성하는 데 필요한 모든 정적 정보를 포함하고 있으며, 이는 JAX의 `PyTreeDef`에 해당합니다.

In [9]:
graphdef, state = nnx.split(model)
# graphdef, state, other = nnx.split(model, nnx.params, ...)
nnx.display(graphdef, state)

### Split, Merge, and Update

`merge`는 `split`의 반대입니다. 이는 `GraphDef`와 `State`를 받아 모듈을 재구성합니다. 아래 예제에서 보이는 것처럼, `split`과 `merge`를 연속적으로 사용하여 어떤 모듈이든 JAX transform에서 사용할 수 있도록 리프팅할 수 있습니다. `update`는 주어진 State의 내용으로 객체를 inplace에서 업데이트할 수 있습니다. 이 패턴은 변환에서 상태를 원본 객체로 다시 전파하는 데 사용됩니다.

In [16]:
print(f'{model.count.value = }')

# 1. Use split to create a pytree representation of the Module
graphdef, state = nnx.split(model)

@jax.jit
def forward(graphdef: nnx.GraphDef, state: nnx.State, x: jax.Array) -> tuple[jax.Array, nnx.State]:
  # 2. Use merge to create a new model inside the JAX transformation
  model = nnx.merge(graphdef, state)
  # 3. Call the Module
  y = model(x)
  # 4. Use split to propagate State updates
  # nnx.split(graph_node) -> (GraphDef, GraphState)
  _, state = nnx.split(model)
  return y, state


# y는 어디다가 쓸까?
y, state = forward(graphdef, state, x=jnp.ones((1, 3)))
# 5. Update the state of the original Module
nnx.update(model, state)

print(f'{model.count.value = }')

model.count.value = Array(1, dtype=uint32)
model.count.value = Array(2, dtype=uint32)


이 패턴의 주요 통찰력은 변환 컨텍스트(기본 eager 인터프리터 포함) 내에서 mutable reference를 사용하는 것이 괜찮지만, 경계를 넘을 때는 Functional API를 사용하는 것이 필요하다는 것입니다.

**왜 모듈이 Pytrees가 아닌가요?** 주된 이유는 이렇게 하면 shared references를 실수로 잃어버리기 매우 쉽기 때문입니다. 예를 들어, 두 개의 모듈이 공유된 모듈을 가지고 있는 경우 JAX boundary를 통해 전달하면 조용히 그 공유를 잃게 됩니다. Functional API는 이 동작을 명시적으로 만들기 때문에 이를 이해하기 훨씬 더 쉽습니다.

### Fine-grained State Control

경험 많은 Linen 및 Haiku 사용자들은 모든 상태를 단일 구조에 갖는 것이 항상 최선의 선택은 아니라는 것을 인식할 수 있습니다. 상태의 다른 하위 집합을 다르게 처리해야 하는 경우가 있기 때문입니다. 예를 들어, JAX transform과 상호 작용할 때 모든 모델의 상태가 분화되거나 해야 할 필요가 없는 경우도 있고, `scan`을 사용할 때 모델 상태의 어느 부분이 전달되고 어느 부분이 아닌지를 지정해야 할 필요가 있는 경우도 있습니다.

이를 해결하기 위해, `split`은 하나 이상의 `Filter`를 전달하여 변수들을 상호 배타적인 상태로 분할할 수 있습니다. 아래에 표시된 것처럼 가장 일반적인 Filter 유형이 있습니다.

In [13]:
# use Variable type filters to split into multiple States
graphdef, params, counts = nnx.split(model, nnx.Param, Count)

nnx.display(params, counts)

In [14]:
# merge multiple States
model = nnx.merge(graphdef, params, counts)
# update with multiple States
nnx.update(model, params, counts)